# Metadata

**L1 Taxonomy** - Backend Development

**L2 Taxonomy** - API Gateways

**Subtopic** - Adding request transformation (URI rewrite or header addition) in a Python API Gateway

**Use Case** - Develop a Python script that simulates an API Gateway. The script should be capable of receiving HTTP requests, rewriting the URI, and adding custom headers before forwarding the request to the intended destination. The destination should be a mock endpoint within the same script. The script should also handle the response from the mock endpoint, removing any added headers before sending the response back to the original requester.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt
Problem Statement:
- Develop a Python script that simulates an API Gateway.
- The script should be capable of receiving HTTP requests, rewriting the URI, and adding custom headers before forwarding the request to the intended destination.
- The destination must be a mock endpoint implemented within the same script.
- "Forwarding" should be handled internally via method calls, not external HTTP requests.
- The script should also handle the response from the mock endpoint, removing any added gateway-specific headers and metadata before sending the response back to the original requester.

Input Format:
- HTTP Method: GET, POST, PUT, or DELETE
- Request URL: /gateway/<path>
- Headers (example):
  - Host: localhost
  - X-Client-ID: <client_id> (optional)
  - Content-Type: application/json (for POST/PUT requests)
- Request Body (if applicable): JSON

Example Request:
Method: POST
URL: /gateway/api/resource
Headers:
```python
    X-Client-ID: test-client
    Content-Type: application/json
Body:
{
    "message": "hello"
}
```
cURL Example:
```bash
curl -X POST http://localhost:8000/gateway/api/resource \
     -H "X-Client-ID: test-client" \
     -H "Content-Type: application/json" \
     -d '{"message": "hello"}'
```

Input Constraints:
- HTTP Method Constraints
  - Must be one of: GET, POST, PUT, DELETE
  - Other methods (e.g. PATCH, OPTIONS) are not supported

- Request URL Format
  - Must start with /gateway/
  - Example: /gateway/api/resource
  - The part after /gateway/ must be internally rewritten to /mock/

- Header Constraints
  - X-Client-ID (optional): Alphanumeric string, max length 64 characters
  - Content-Type: Required for methods with body, must be application/json
  - Beyond Host, X-Client-ID, and Content-Type, any other client-provided headers must be removed by the gateway before forwarding the request to the mock endpoint.

- Request Body Constraints
  - Allowed only for POST and PUT methods
  - Must be a valid JSON object (not array or primitive)
  - Maximum size: 10 KB - requests exceeding this must return 413 Payload Too Large

- Character Encoding
  - All input must be encoded in UTF-8

- Concurrency / Rate Limiting (Optional Feature)
  - For simulation purposes: assume a maximum of 10 concurrent requests

Output Format:
- The response returned by the API Gateway should include:
  - Status Code: 200 OK (or appropriate 4xx/5xx error)
  - Headers:
    - Content-Type: application/json
    - (Other headers from the mock endpoint, excluding gateway-specific headers)
  - Body: JSON object returned by the mock endpoint

- The API Gateway must remove all internal headers (e.g., X-Gateway-Processed, X-Forwarded-For) and any gateway-specific metadata from the response before sending it to the client.

Example Successful Response:
```python
Status: 200 OK
Headers:
  Content-Type: application/json
Body:
{
  "status": "success",
  "message": "Processed by mock endpoint",
  "path": "/mock/api/resource",
  "client_id": "test-client"
}

Example Error Response (Invalid Method):
Status: 405 Method Not Allowed
Headers:
  Content-Type: application/json
Body:
{
  "error": "Method not allowed"
}

Example Error Response (Invalid JSON):
Status: 400 Bad Request
Headers:
  Content-Type: application/json
Body:
{
  "error": "Invalid JSON payload"
}
```

Class Definition and Function Signature:
```python
class APIGatewayHandler(http.server.BaseHTTPRequestHandler):
    """
    A request handler that simulates an API Gateway.
    It rewrites URIs, adds custom headers, forwards to a mock endpoint
    internally (not over the network), and removes gateway-related data from the response.
    """

    def do_GET(self):
        """Handle GET requests."""
        pass

    def do_POST(self):
        """Handle POST requests."""
        pass

    def do_PUT(self):
        """Handle PUT requests."""
        pass

    def do_DELETE(self):
        """Handle DELETE requests."""
        pass

    def _rewrite_path(self, path: str) -> str:
        """
        Rewrite incoming /gateway/... path to /mock/... for internal forwarding.
        """
        pass

    def _add_gateway_headers(self, headers: dict) -> dict:
        """
        Add custom headers (e.g., X-Gateway-Processed) before internal forwarding.
        """
        pass

    def _remove_gateway_headers(self, headers: dict) -> dict:
        """
        Remove gateway-specific headers from the final response to the client.
        """
        pass

    def _handle_mock_endpoint(self, method: str, path: str, headers: dict, body: bytes) -> tuple[int, dict, bytes]:
        """
        Simulate processing by the mock backend service.

        Returns:
            A tuple of (status_code, response_headers, response_body)
        """
        pass
```

# Requirements
Explicit Requirements:
- The script must receive and process HTTP requests (GET, POST, PUT, DELETE)
- It must rewrite incoming URIs from /gateway/... to /mock/... before forwarding
- The API Gateway must add custom headers (e.g., X-Gateway-Processed) before forwarding
- The forwarded request should be internally handled by a mock endpoint in the same script
- The response must have gateway-specific headers and metadata removed before returning to the client

Implicit Requirements:
- The request body (if present) must be valid JSON and UTF-8 encoded
- Only requests starting with /gateway/ should be processed; others must return 404
- The gateway must correctly handle invalid input (e.g., unsupported methods, malformed JSON)
- The system should log internal processing steps (optional, but helpful for debugging)
- The mock endpoint should echo or generate a response based on the request path or content

Solution Expectations:
- The script should start a local HTTP server (e.g., using http.server or socketserver)
- It should define a request handler class (e.g., APIGatewayHandler)
- The handler must support GET, POST, PUT, and DELETE methods
- All requests to /gateway/... must be internally rewritten to /mock/...
- The handler should:
  - Parse headers and body (if any)
  - Add internal headers like X-Gateway-Processed
  - Forward the request internally to a method simulating the mock service
- The mock endpoint must:
  - Simulate real backend logic (e.g., echo path or client_id)
- The API Gateway must:
  - Remove gateway-specific headers and metadata before responding
  - Return appropriate status codes (200, 400, 404, 405, 413, etc.)
- All request/response bodies must be in UTF-8 encoded JSON
- The script must be self-contained and executable with a main block

Edge Cases and Expected Behavior:
- Request to a non-/gateway/ path
  - Response: 404 Not Found

- Invalid HTTP method (e.g., PATCH, OPTIONS)
  - Response: 405 Method Not Allowed

- POST or PUT request with invalid JSON body
  - Response: 400 Bad Request

- Missing or malformed headers
  - Proceed with defaults where applicable (e.g., no client ID)

- Large request body exceeding 10 KB
  - Response: 413 Payload Too Large

- GET request with query parameters
  - Preserve query string in URI rewrite to /mock/

- Gateway-injected headers not removed from final response
  - Must be stripped before returning to client

- Unknown /mock/... path (after rewrite)
  - Response: 404 Not Found

- Concurrent requests (optional)
  - Must isolate each request context properly

Solution Constraints:
- Only standard Python libraries may be used (http.server, socketserver, json, etc.)
- The server must run locally on a fixed port (e.g., 8000)
- Script must be self-contained and executable without external dependencies
- URI rewriting from /gateway/... to /mock/... must be strict and internal
- All request and response bodies must be UTF-8 encoded JSON
- Gateway logic must not leak internal headers or metadata
- Errors must return proper status codes and JSON-formatted error messages
- The mock endpoint must be implemented inside the same script (no external API calls)

In [ ]:
# code
"""API Gateway simulation for HTTP server requests."""

import http.server
import socketserver
import json
import re
from urllib.parse import urlparse

PORT = 8000
MAX_REQUEST_BODY_SIZE = 10 * 1024


class APIGatewayHandler(http.server.BaseHTTPRequestHandler):
    """Handles HTTP requests with simulated API Gateway behavior."""

    SUPPORTED_METHODS = ['GET', 'POST', 'PUT', 'DELETE']
    GATEWAY_PREFIX = '/gateway/'
    MOCK_PREFIX = '/mock/'
    GATEWAY_INTERNAL_HEADERS = ['X-Gateway-Processed', 'X-Forwarded-For']
    ALLOWED_CLIENT_HEADERS = ['Host', 'X-Client-ID', 'Content-Type']

    def _send_error_response(self, status_code: int, message: str):
        """Send a JSON-formatted error response."""
        self.send_response(status_code)
        self.send_header('Content-Type', 'application/json')
        self.end_headers()
        response_body = json.dumps({"error": message}).encode('utf-8')
        self.wfile.write(response_body)

    def _process_request(self, method: str):
        """Parse and validate the request and delegate to the mock handler."""
        parsed_url = urlparse(self.path)
        path = parsed_url.path
        query_string = parsed_url.query

        if not path.startswith(self.GATEWAY_PREFIX):
            self._send_error_response(404, "Not Found")
            return

        incoming_headers = {k: v for k, v in self.headers.items()}
        filtered_headers = {
            h: incoming_headers[h] for h in self.ALLOWED_CLIENT_HEADERS
            if h in incoming_headers
        }

        if 'X-Client-ID' in filtered_headers:
            client_id = filtered_headers['X-Client-ID']
            if not isinstance(client_id, str) or not re.fullmatch(
                r'^[a-zA-Z0-9]{1,64}$', client_id
            ):
                self._send_error_response(
                    400, "Invalid X-Client-ID header format or length."
                )
                return

        request_body = b''
        if method in ['POST', 'PUT']:
            content_length = int(self.headers.get('Content-Length', 0))
            if content_length > MAX_REQUEST_BODY_SIZE:
                self._send_error_response(413, "Payload Too Large")
                return
            if (
                'Content-Type' not in filtered_headers or
                filtered_headers['Content-Type'] != 'application/json'
            ):
                self._send_error_response(
                    400,
                    "Content-Type must be"
                    " application/json for POST/PUT requests."
                )
                return
            try:
                request_body = self.rfile.read(content_length)
                if request_body:
                    decoded_body = json.loads(request_body.decode('utf-8'))
                    if not isinstance(decoded_body, dict):
                        self._send_error_response(
                            400, "Request body must be a valid JSON object."
                        )
                        return
            except json.JSONDecodeError:
                self._send_error_response(400, "Invalid JSON payload.")
                return
            except UnicodeDecodeError:
                self._send_error_response(
                    400, "Request body must be UTF-8 encoded."
                )
                return
        elif int(self.headers.get('Content-Length', 0)) > 0:
            self._send_error_response(
                400, "Request body not allowed for this method."
            )
            return

        rewritten_path = self._rewrite_path(path)
        if query_string:
            rewritten_path += '?' + query_string

        forwarded_headers = self._add_gateway_headers(filtered_headers)
        status_code, mock_response_headers, mock_response_body = (
            self._handle_mock_endpoint(
                method, rewritten_path, forwarded_headers, request_body
            )
        )
        final_headers = self._remove_gateway_headers(mock_response_headers)
        self.send_response(status_code)
        for header, value in final_headers.items():
            self.send_header(header, value)
        self.end_headers()
        self.wfile.write(mock_response_body)

    def do_get(self):
        """Handle GET requests."""
        self._process_request('GET')

    def do_post(self):
        """Handle POST requests."""
        self._process_request('POST')

    def do_put(self):
        """Handle PUT requests."""
        self._process_request('PUT')

    def do_delete(self):
        """Handle DELETE requests."""
        self._process_request('DELETE')

    def do_head(self):
        """Reject unsupported HEAD requests."""
        self._send_error_response(405, "Method not allowed")

    def do_options(self):
        """Reject unsupported OPTIONS requests."""
        self._send_error_response(405, "Method not allowed")

    def _rewrite_path(self, path: str) -> str:
        """Rewrite path from gateway to mock prefix."""
        return path.replace(self.GATEWAY_PREFIX, self.MOCK_PREFIX, 1)

    def _add_gateway_headers(self, headers: dict) -> dict:
        """Add internal headers to simulate gateway forwarding."""
        modified_headers = headers.copy()
        modified_headers['X-Gateway-Processed'] = 'true'
        modified_headers['X-Forwarded-For'] = self.client_address[0]
        return modified_headers

    def _remove_gateway_headers(self, headers: dict) -> dict:
        """Remove internal headers before response."""
        return {
            k: v for k, v in headers.items()
            if k not in self.GATEWAY_INTERNAL_HEADERS
        }

    def _handle_mock_endpoint(
        self, method: str, path: str, headers: dict, body: bytes
    ) -> tuple[int, dict, bytes]:
        """Simulate the internal mock endpoint logic."""
        print(
            f"Mock Endpoint Received: Method={method}, Path={path}, "
            f"Headers={headers}, Body={body.decode('utf-8', errors='ignore')}"
        )
        status_code = 200
        response_headers = {'Content-Type': 'application/json'}
        response_body_data = {
            "status": "success",
            "message": "Processed by mock endpoint",
            "method": method,
            "path": path,
            "gateway_processed": headers.get('X-Gateway-Processed'),
            "client_id": headers.get('X-Client-ID', 'N/A')
        }

        parsed_url = urlparse(path)
        mock_path_segments = parsed_url.path.split('/')

        if (len(mock_path_segments) > 2
                and mock_path_segments[2] == 'nonexistent'):
            status_code = 404
            response_body_data = {
                "error": "Resource not found at mock endpoint"
            }
        elif method in ['POST', 'PUT'] and body:
            try:
                decoded_body = json.loads(body.decode('utf-8'))
                response_body_data["received_payload"] = decoded_body
            except json.JSONDecodeError:
                status_code = 400
                response_body_data = {
                    "error": "Invalid JSON received by mock endpoint"
                }

        response_headers['X-Mock-Service-Header'] = 'true'
        response_headers['X-Gateway-Processed'] = 'true-from-mock'

        return (
            status_code,
            response_headers,
            json.dumps(response_body_data).encode('utf-8')
        )


class ThreadedHTTPServer(socketserver.ThreadingMixIn, http.server.HTTPServer):
    """Threaded HTTP server with daemon threads enabled."""

    daemon_threads = True


if __name__ == "__main__":
    print(f"Starting API Gateway on port {PORT}...")
    try:
        with ThreadedHTTPServer(("", PORT), APIGatewayHandler) as httpd:
            print(f"Server running on http://localhost:{PORT}")
            httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down API Gateway.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [ ]:
# tests

"""Test suite for API Gateway Handler functionality."""

import unittest
import json
import threading
import time
import socket
from http.client import HTTPConnection
from main import APIGatewayHandler
import socketserver


class ReuseAddrTCPServer(socketserver.TCPServer):
    """TCPServer that enables socket reuse for Docker environments."""

    allow_reuse_address = True

    def server_bind(self):
        """Configure socket for address reuse."""
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        super().server_bind()


class TestAPIGateway(unittest.TestCase):
    """Test suite for API Gateway functionality."""

    @classmethod
    def setUpClass(cls):
        """Start the test server once for all tests."""
        cls.server_port = 8001

        for attempt in range(3):
            try:
                cls.server = ReuseAddrTCPServer(
                    ("", cls.server_port), APIGatewayHandler
                )
                break
            except OSError as e:
                if e.errno == 98 and attempt < 2:
                    time.sleep(1)
                    continue
                raise

        cls.server_thread = threading.Thread(target=cls.server.serve_forever)
        cls.server_thread.daemon = True
        cls.server_thread.start()
        time.sleep(0.5)

    @classmethod
    def tearDownClass(cls):
        """Stop the test server."""
        try:
            cls.server.shutdown()
            cls.server.server_close()
            time.sleep(0.1)
        except Exception:
            pass

    def setUp(self):
        """Set up HTTP connection for each test."""
        self.conn = HTTPConnection("localhost", self.server_port)

    def tearDown(self):
        """Close HTTP connection after each test."""
        self.conn.close()

    def _make_request(self, method, path, headers=None, body=None):
        """Make HTTP requests."""
        if headers is None:
            headers = {}

        if body is not None and isinstance(body, dict):
            body = json.dumps(body)
            if "Content-Type" not in headers:
                headers["Content-Type"] = "application/json"

        self.conn.request(method, path, body, headers)
        response = self.conn.getresponse()
        response_body = response.read().decode("utf-8")

        return {
            "status": response.status,
            "headers": dict(response.headers),
            "body": json.loads(response_body) if response_body else None,
        }

    def test_get_request_success(self):
        """Test successful GET request with path rewriting."""
        headers = {"X-Client-ID": "test-client-123"}
        response = self._make_request("GET", "/gateway/api/users", headers)

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users")
        self.assertEqual(response["body"]["method"], "GET")
        self.assertEqual(response["body"]["client_id"], "test-client-123")
        self.assertEqual(response["body"]["status"], "success")

    def test_post_request_with_json_body(self):
        """Test POST request with valid JSON body."""
        headers = {
            "X-Client-ID": "test-client",
            "Content-Type": "application/json",
        }
        body = {"message": "hello", "user": "john"}
        response = self._make_request(
            "POST", "/gateway/api/resource", headers, body
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/resource")
        self.assertEqual(response["body"]["method"], "POST")
        self.assertEqual(response["body"]["client_id"], "test-client")
        self.assertEqual(response["body"]["received_data"], body)

    def test_put_request_with_headers(self):
        """Test PUT request with JSON body and custom headers."""
        headers = {
            "Content-Type": "application/json",
            "X-Client-ID": "client-456",
        }
        body = {"name": "Updated User", "age": 30}
        response = self._make_request(
            "PUT", "/gateway/api/users/123", headers, body
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users/123")
        self.assertEqual(response["body"]["method"], "PUT")
        self.assertEqual(response["body"]["client_id"], "client-456")
        self.assertEqual(response["body"]["received_data"], body)

    def test_delete_request(self):
        """Test DELETE request handling."""
        headers = {"X-Client-ID": "admin-client"}
        response = self._make_request(
            "DELETE", "/gateway/api/users/456", headers
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(response["body"]["path"], "/mock/api/users/456")
        self.assertEqual(response["body"]["method"], "DELETE")
        self.assertEqual(response["body"]["client_id"], "admin-client")

    def test_invalid_path_not_gateway(self):
        """Test request to path not starting with /gateway/."""
        response = self._make_request("GET", "/api/users")

        self.assertEqual(response["status"], 404)
        self.assertEqual(response["body"]["error"], "Not found")

    def test_invalid_method_patch(self):
        """Test unsupported HTTP method (PATCH)."""
        response = self._make_request("PATCH", "/gateway/api/resource")

        self.assertEqual(response["status"], 405)
        self.assertEqual(response["body"]["error"], "Method not allowed")

    def test_invalid_json_body(self):
        """Test POST with malformed JSON."""
        headers = {"Content-Type": "application/json"}
        self.conn.request(
            "POST", "/gateway/api/data", "{invalid json}", headers
        )
        response = self.conn.getresponse()
        response_body = json.loads(response.read().decode("utf-8"))

        self.assertEqual(response.status, 400)
        self.assertEqual(response_body["error"], "Invalid JSON payload")

    def test_oversized_payload(self):
        """Test request body exceeding 10KB limit."""
        headers = {"Content-Type": "application/json"}
        large_data = {"data": "x" * 11000}
        response = self._make_request(
            "POST", "/gateway/api/upload", headers, large_data
        )

        self.assertEqual(response["status"], 413)
        self.assertEqual(response["body"]["error"], "Payload too large")

    def test_get_with_query_parameters(self):
        """Test GET request with query string preservation."""
        response = self._make_request(
            "GET", "/gateway/api/search?q=test&limit=10&page=2"
        )

        self.assertEqual(response["status"], 200)
        self.assertEqual(
            response["body"]["path"], "/mock/api/search?q=test&limit=10&page=2"
        )
        self.assertEqual(response["body"]["method"], "GET")

    def test_gateway_headers_removed(self):
        """Verify gateway-specific headers are removed from response."""
        headers = {"X-Client-ID": "test-verify-headers"}
        response = self._make_request("GET", "/gateway/api/headers", headers)

        self.assertEqual(response["status"], 200)
        self.assertNotIn("X-Gateway-Processed", response["headers"])
        self.assertNotIn("X-Forwarded-For", response["headers"])

    def test_post_without_content_type(self):
        """Test POST request missing Content-Type header."""
        self.conn.request("POST", "/gateway/api/data", '{"test": "data"}')
        response = self.conn.getresponse()
        response_body = json.loads(response.read().decode("utf-8"))

        self.assertEqual(response.status, 400)
        self.assertEqual(
            response_body["error"],
            "Content-Type required for POST/PUT requests",
        )

    def test_empty_body_post(self):
        """Test POST with empty body."""
        headers = {"Content-Type": "application/json"}
        response = self._make_request(
            "POST", "/gateway/api/empty", headers, ""
        )

        self.assertEqual(response["status"], 400)
        self.assertEqual(
            response["body"]["error"], "Empty body not allowed for POST/PUT"
        )


if __name__ == "__main__":
    unittest.main(verbosity=2)


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/fbda8836-c674-4d23-8ac9-923f396af119?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74791%2Fview

#### Model code:

```python
# code generated by the model
import http.server
import json
import urllib.parse

MAX_BODY_SIZE = 10 * 1024  # 10 KB

class APIGatewayHandler(http.server.BaseHTTPRequestHandler):
    def _send_response(self, status_code, headers, body):
        self.send_response(status_code)
        for key, value in headers.items():
            self.send_header(key, value)
        self.end_headers()
        if body:
            self.wfile.write(body)

    def do_get(self):
        self._process_request()

    def do_post(self):
        self._process_request()

    def do_put(self):
        self._process_request()

    def do_delete(self):
        self._process_request()

    def _process_request(self):
        if not self.path.startswith("/gateway/"):
            return self._send_error_response(404, "Not found")
        
        # Validate method
        if self.command not in ['GET', 'POST', 'PUT', 'DELETE']:
            return self._send_error_response(405, "Method not allowed")

        # Rewrite path
        rewritten_path = self._rewrite_path(self.path)

        # Get headers with only allowed headers
        headers = self._filter_request_headers(self.headers)
        headers = self._add_gateway_headers(headers)

        # Handle body
        content_length = int(self.headers.get('Content-Length', 0))
        if content_length > MAX_BODY_SIZE:
            return self._send_error_response(413, "Payload too large")

        body = self.rfile.read(content_length) if content_length else None

        # Validate JSON body if it's POST or PUT
        if self.command in ['POST', 'PUT']:
            if not self.headers.get('Content-Type') == 'application/json':
                return self._send_error_response(400, "Content-Type must be application/json")
            
            try:
                json_body = json.loads(body) if body else {}
                if not isinstance(json_body, dict):
                    raise ValueError()
            except (json.JSONDecodeError, ValueError):
                return self._send_error_response(400, "Invalid JSON payload")

        # Forward request to the mock endpoint
        status, response_headers, response_body = self._handle_mock_endpoint(self.command, rewritten_path, headers, body)

        # Send the response back to client, stripping gateway headers
        final_headers = self._remove_gateway_headers(response_headers)
        self._send_response(status, final_headers, response_body)

    def _rewrite_path(self, path: str) -> str:
        return path.replace("/gateway/", "/mock/", 1)

    def _add_gateway_headers(self, headers: dict) -> dict:
        headers["X-Gateway-Processed"] = "true"
        return headers

    def _remove_gateway_headers(self, headers: dict) -> dict:
        headers.pop("X-Gateway-Processed", None)
        return headers

    def _filter_request_headers(self, headers) -> dict:
        filtered_headers = {}
        for key in headers:
            if key in ["Host", "X-Client-ID", "Content-Type"]:
                filtered_headers[key] = headers[key]
        return filtered_headers

    def _send_error_response(self, status_code, message):
        error_response = {
            "error": message
        }
        self._send_response(status_code, {"Content-Type": "application/json"}, json.dumps(error_response).encode())

    def _handle_mock_endpoint(self, method: str, path: str, headers: dict, body: bytes) -> tuple:
        # Mock response
        # <Issue>: Accepts any rewritten path under /mock/... and returns a 200 OK, regardless of whether the path is "known" or valid.
        response = {
            "status": "success",
            "message": "Processed by mock endpoint",
            "path": path,
            "client_id": headers.get("X-Client-ID", None)
        }
        return 200, {"Content-Type": "application/json"}, json.dumps(response).encode()

def run(server_class=http.server.HTTPServer, handler_class=APIGatewayHandler, port=8000):
    server_address = ('', port)
    httpd = server_class(server_address, handler_class)
    print(f'Starting httpd server on port {port}')
    httpd.serve_forever()

if __name__ == "__main__":
    run()
```